In [55]:
import openai
from PIL import Image, ImageDraw, ImageFont
import datetime
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from pathlib import Path
import random # Added for random symbol selection
import matplotlib.offsetbox as offsetbox # Added for image embedding
import matplotlib.image as mpimg # Added for loading images in matplotlib

In [56]:
# --- Configuration ---
# Replace with your actual key if running the full main() function
openai.api_key = "OPENAI_KEY" 
# Define the folder where symbol images are stored
SYMBOLS_FOLDER = "sports_symbols"

In [57]:
# --- Dummy Data Function ---
def create_dummy_data():
    """Creates dummy event data for roadmap testing."""
    dummy_events = [
        {
            "time": "10:00",
            "activity": "Football Match: A vs B",
            "location": "Main Stadium",
            "description": "Watch the exciting clash between Team A and Team B. Expect a large crowd and lots of action.",
            "type": "football" # Added for potential future symbol matching
        },
        {
            "time": "14:30",
            "activity": "Table Tennis Fun",
            "location": "Community Sports Hall",
            "description": "Casual table tennis session with friends. A good way to relax and have some fun.",
            "type": "table-tennis" # Added for potential future symbol matching
        },
        {
            "time": "17:00",
            "activity": "Judo Trial Class",
            "location": "Martial Arts Center",
            "description": "Give Judo a try! Learn basic throws and holds in this introductory session.",
            "type": "judo" # Added for potential future symbol matching
        }
    ]
    # The function create_day_roadmap expects a list of events
    return dummy_events

In [58]:
# --- Original Functions (Input Collection, OpenAI Interaction) ---
# These functions remain unchanged but are kept for completeness 
# if you want to run the full workflow later.

def collect_user_inputs():
    """Collect comprehensive user inputs for the sports event trip planning"""
    print("\n===== SPORTS EVENT TRIP PLANNER =====")
    print("Let's create your personalized sports event journey roadmap!\n")

    start_date = input("Enter trip start date (YYYY-MM-DD, e.g., 2025-03-31): ")
    try:
        start_date_obj = datetime.datetime.strptime(start_date, '%Y-%m-%d')
    except ValueError:
        print("Invalid start date format. Using today's date as default.")
        start_date_obj = datetime.datetime.now()
        start_date = start_date_obj.strftime('%Y-%m-%d')

    end_date = input("Enter trip end date (YYYY-MM-DD, e.g., 2025-04-02): ")
    try:
        end_date_obj = datetime.datetime.strptime(end_date, '%Y-%m-%d')
        if end_date_obj < start_date_obj:
            raise ValueError("End date must be after start date.")
    except ValueError as e:
        print(f"Invalid end date: {e}. Using 3 days from start date as default.")
        end_date_obj = start_date_obj + datetime.timedelta(days=3)
        end_date = end_date_obj.strftime('%Y-%m-%d')

    duration = (end_date_obj - start_date_obj).days + 1

    location = input("Enter event city and country (e.g., 'Riyadh, Saudi Arabia'): ")
    event_type = input("What type of sports event are you attending? (e.g., football, F1, tennis): ")
    event_name = input("Enter specific event name if known (e.g., 'FIFA World Cup', 'Saudi Grand Prix'): ")

    transport_mode = input("How will you arrive at your destination? (e.g., flight, train, drive): ")
    accommodation = input("Where will you be staying? (e.g., hotel name, Airbnb, staying with friends): ")
    accommodation_location = input("In which area/neighborhood is your accommodation? (e.g., downtown, near stadium): ")

    has_ticket = input("Do you have tickets for any events already? (yes/no): ").lower() == 'yes'
    ticket_details = []
    if has_ticket:
        num_tickets = int(input("How many different event tickets do you have? "))
        for i in range(num_tickets):
            print(f"\nTicket #{i+1}:")
            event = input("  Event name: ")
            date = input("  Date (YYYY-MM-DD): ")
            time = input("  Time (e.g., 19:30): ")
            venue = input("  Venue: ")
            seat = input("  Seat info (optional): ")
            ticket_details.append({"event": event, "date": date, "time": time, "venue": venue, "seat": seat})
    else:
        ticket_interest = input("Are you interested in getting tickets for any specific events? Please describe: ")

    print("\nPlease rate your interest in the following activities (1-5, where 5 is highest):")
    food_interest = int(input("Local cuisine exploration (1-5): ") or "3")
    sightseeing_interest = int(input("Sightseeing & tourism (1-5): ") or "3")
    fanzone_interest = int(input("Fan zones & meetups (1-5): ") or "3")
    shopping_interest = int(input("Shopping (1-5): ") or "2")
    nightlife_interest = int(input("Nightlife & entertainment (1-5): ") or "2")

    dietary_restrictions = input("Any dietary restrictions? ")
    mobility_limitations = input("Any mobility limitations to consider? ")
    budget_level = input("Budget level (economy, mid-range, luxury): ").lower() or "mid-range"
    special_requests = input("Any other special requests or preferences? ")

    return {
        "start_date": start_date,
        "end_date": end_date,
        "duration": duration,
        "location": location,
        "event_type": event_type,
        "event_name": event_name,
        "transport_mode": transport_mode,
        "accommodation": accommodation,
        "accommodation_location": accommodation_location,
        "has_ticket": has_ticket,
        "ticket_details": ticket_details if has_ticket else None,
        "ticket_interest": ticket_interest if not has_ticket else None,
        "preferences": {
            "food": food_interest,
            "sightseeing": sightseeing_interest,
            "fanzones": fanzone_interest,
            "shopping": shopping_interest,
            "nightlife": nightlife_interest
        },
        "dietary_restrictions": dietary_restrictions,
        "mobility_limitations": mobility_limitations,
        "budget_level": budget_level,
        "special_requests": special_requests
    }

In [59]:
def generate_itinerary(inputs):
    """Generate detailed itinerary with explicit TIME field included"""
    # This function uses OpenAI - ensure API key is set if you run this part
    openai.api_key = os.getenv("OPENAI_API_KEY", "YOUR_DEFAULT_KEY_HERE") # Safer way to handle key
    if openai.api_key == "YOUR_DEFAULT_KEY_HERE":
         print("Warning: OpenAI API key not set. Itinerary generation will likely fail.")
         # You might want to return a dummy response or raise an error here
         # For now, let's just print the warning.

    ticket_info = "N/A"
    if inputs["has_ticket"] and inputs["ticket_details"]:
        ticket_info = "\n".join([
            f"- Event: {t['event']}, Date: {t['date']}, Time: {t['time']}, Venue: {t['venue']}, Seat: {t['seat']}"
            for t in inputs["ticket_details"]
        ])

    preference_ratings = []
    for pref, rating in inputs["preferences"].items():
        stars = "★" * rating + "☆" * (5 - rating)
        preference_ratings.append(f"{pref.capitalize()}: {stars} ({rating}/5)")
    preference_info = "\n".join(preference_ratings)

    prompt = f"""
Create a detailed sports event roadmap for a fan with the following details:

BASIC INFORMATION:
- Trip dates: {inputs['start_date']} to {inputs['end_date']} ({inputs['duration']} days)
- Location: {inputs['location']}
- Event type: {inputs['event_type']}
- Event name: {inputs['event_name']}

LOGISTICS:
- Transport mode: {inputs['transport_mode']}
- Accommodation: {inputs['accommodation']} at {inputs['accommodation_location']}

TICKETS:
- Has tickets: {'Yes' if inputs['has_ticket'] else 'No'}
- Ticket details: {ticket_info}
{f"- Ticket interests: {inputs['ticket_interest']}" if not inputs['has_ticket'] else ""}

PREFERENCES:
{preference_info}
- Dietary restrictions: {inputs['dietary_restrictions'] or 'None'}
- Mobility limitations: {inputs['mobility_limitations'] or 'None'}
- Budget level: {inputs['budget_level']}
- Special requests: {inputs['special_requests'] or 'None'}

INSTRUCTIONS:
1. Create a detailed daily itinerary for the trip duration.
2. Include arrival and departure logistics.
3. Incorporate the sports event(s) and mix in other activities based on preferences.
4. Assign approximate times where relevant (e.g., '14:00', '19:30').
5. Keep descriptions concise but informative.

Format each entry as:
NUMBER||EVENT_NAME||EVENT_DESCRIPTION||CATEGORY||DATE||TIME

Categories: SPORT, SHOPPING, FOOD, SIGHTSEEING, EVENT, TRANSPORT, ENTERTAINMENT, FAN_EXPERIENCE, ACCOMMODATION, OTHER

Example:
1||Arrival in Riyadh||Flight landing at King Khalid International||TRANSPORT||2025-03-31||14:00
2||Saudi Grand Prix||F1 race at Jeddah Corniche Circuit||SPORT||2025-04-01||19:30
"""

    try:
        response = openai.ChatCompletion.create(
            model="gpt-4-turbo", # Or your preferred model
            messages=[{"role": "user", "content": prompt}],
            max_tokens=1500
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error calling OpenAI: {e}")
        return f"Error generating itinerary: {e}" # Return error message

In [75]:

def generate_roadmap_visual(num_nodes=5, node_labels=None, event_details=None, 
                            filename="roadmap_visual.png", symbol_folder=None, 
                            event_types=None): # Added symbol_folder and event_types
    """
    Generates a horizontal roadmap visualization with embedded symbols in nodes.
      - Nodes are aligned horizontally.
      - Alternating curved dashed arcs connect the nodes.
      - Each node ends with a small numbered circle.
      - **MODIFIED:** Embeds a symbol (icon) in the center of each main node.
      - If provided, node_labels (list of strings) are shown relative to the circle:
            * For nodes where the time is drawn above, event details appear in a box below.
            * For nodes where the time is drawn below, event details appear in a box above.
    
    Args:
        num_nodes (int): Number of nodes.
        node_labels (list of str, optional): Labels to display (e.g., TIME).
        event_details (list of str, optional): Detailed event text to show in a box.
        filename (str): Where to save the image.
        symbol_folder (str, optional): Path to the folder containing symbol PNGs.
        event_types (list of str, optional): List of types corresponding to events 
                                            (used for future symbol mapping, currently ignored).
    """
    # --- Configuration ---
    node_radius = 0.32
    x_spacing = 2.8 # Increased spacing slightly more for symbols
    arc_height = 0.5
    small_circle_radius = 0.1 
    counter_line_offset = 0.2
    node_colors = ['#2AC096', '#282F73'] # old ['#6053D3', '#292263']
    arc_line_color = 'black'
    line_style = '--'
    background_color = '#E0E0E0'
    number_color = 'white'
    number_fontsize = 12  
    label_fontsize = 10   
    detail_fontsize = 12  
    label_color = 'black'
    symbol_zoom = 0.60 # **TUNE THIS** factor to scale the 63x63px symbol inside the node 

    # --- Symbol Loading Setup ---
    symbol_files = []
    if symbol_folder and os.path.isdir(symbol_folder):
        try:
            symbol_files = [f for f in os.listdir(symbol_folder) if f.lower().endswith('.png')]
            if not symbol_files:
                print(f"Warning: No PNG files found in symbol folder: {symbol_folder}")
        except Exception as e:
            print(f"Warning: Could not read symbol folder '{symbol_folder}'. Error: {e}")
            symbol_folder = None # Disable symbol loading if folder is inaccessible
    else:
        print("Warning: Symbol folder not provided or not found. Symbols will not be added.")
        symbol_folder = None # Ensure it's None if not valid

    # --- Calculations ---
    main_node_y = 0
    x_coords = np.arange(num_nodes) * x_spacing
    
    # Determine positions for the small numbered circles
    y_coords_small = []
    for i in range(num_nodes):
        dist_to_small_center = node_radius + counter_line_offset + small_circle_radius
        if i % 2 == 0:  # Even index: small circle slightly lower
            small_y = main_node_y - dist_to_small_center + 0.1 * dist_to_small_center
        else:           # Odd index: small circle slightly higher
            small_y = main_node_y + dist_to_small_center - 0.1 * dist_to_small_center
        y_coords_small.append(small_y)
    y_coords_small = np.array(y_coords_small)
    
    # --- Plotting ---
    fig, ax = plt.subplots(figsize=(4 * num_nodes, 8)) # Adjust figsize as needed
    fig.patch.set_facecolor(background_color)
    ax.set_facecolor(background_color)
    
    # Draw alternating arcs between nodes
    for i in range(num_nodes - 1):
        x1, x2 = x_coords[i], x_coords[i+1]
        arc_center_x = (x1 + x2) / 2
        theta1, theta2 = (0, 180) if i % 2 == 0 else (180, 360)
        arc = patches.Arc((arc_center_x, main_node_y), width=x_spacing, height=arc_height,
                          angle=0, theta1=theta1, theta2=theta2,
                          color=arc_line_color, linestyle=line_style, lw=3, zorder=1)
        ax.add_patch(arc)
    
    # Draw nodes, symbols, lines, circles, and labels
    for i in range(num_nodes):
        node_color = node_colors[i % 2]
        x = x_coords[i]
        small_y = y_coords_small[i]
        
        # Draw vertical dashed line
        ax.plot([x, x], [main_node_y, small_y],
                linestyle=line_style, color=node_color, lw=2, zorder=1)
        
        # Draw main node (large circle background)
        main_circle = plt.Circle((x, main_node_y), node_radius, color=node_color, zorder=2)
        ax.add_patch(main_circle)

        # --- Add Symbol ---
        if symbol_folder and symbol_files: # Check again if symbols are usable
            try:
                # --- Random Selection (for testing) ---
                selected_symbol_file = random.choice(symbol_files)
                symbol_path = os.path.join(symbol_folder, selected_symbol_file)

                # --- Future Matching Logic (Placeholder) ---
                # if event_types and i < len(event_types):
                #     event_type = event_types[i]
                #     # Find matching symbol file based on event_type...
                #     # symbol_path = find_symbol_for_type(event_type, symbol_folder, symbol_files)
                
                if os.path.exists(symbol_path):
                    # Load symbol image using matplotlib
                    symbol_img = mpimg.imread(symbol_path) 

                    # Create an OffsetImage for the symbol
                    imagebox = offsetbox.OffsetImage(symbol_img, zoom=symbol_zoom)
                    imagebox.image.axes = ax

                    # Create an AnnotationBbox to place the image at the node center
                    ab = offsetbox.AnnotationBbox(imagebox, (x, main_node_y),
                                                  frameon=False,
                                                  pad=0.0, # No padding around the image itself
                                                  xycoords='data',
                                                  boxcoords="data", # Match coordinate system
                                                  bboxprops=dict(edgecolor='none')) 

                    # Add the annotation box (symbol) to the axes
                    ax.add_artist(ab)
                    # Place symbol above circle (zorder=2) but below number/small circle (zorder=3+)
                    ab.set_zorder(2.5) 
                else:
                     print(f"Warning: Symbol file not found: {symbol_path}")

            except Exception as e:
                print(f"Warning: Could not load or place symbol for node {i+1} ('{symbol_path}'). Error: {e}")
        # --- End Add Symbol ---

        # Draw small numbered circle
        small_circle = plt.Circle((x, small_y), small_circle_radius,
                                  color=node_color, edgecolor=node_color,
                                  lw=2, zorder=3)
        ax.add_patch(small_circle)
        
        # Add node number
        ax.text(x, small_y, str(i + 1), ha='center', va='center',
                fontsize=number_fontsize, color=number_color, zorder=4)
        
        # Draw the time label (if provided) near the main node.
        if node_labels and i < len(node_labels):
            offset = 0.2
            if i % 2 == 0: # Even index: time label above
                time_y = main_node_y + node_radius + offset
                va_time = 'bottom'
            else:          # Odd index: time label below
                time_y = main_node_y - node_radius - offset
                va_time = 'top'
            ax.text(x, time_y, node_labels[i],
                    ha='center', va=va_time, fontsize=label_fontsize,
                    color=label_color, zorder=5, fontweight='bold')
        
        # Draw event details in a text box
        if event_details and i < len(event_details):
            box_offset = 0.6 
            bbox_props = dict(boxstyle="round,pad=0.5", fc="white", ec=node_color, alpha=0.9)
            if i % 2 == 0: # Even index: event box below
                event_y = main_node_y - node_radius - box_offset
                va_event = 'top'
            else:          # Odd index: event box above
                event_y = main_node_y + node_radius + box_offset
                va_event = 'bottom'
            ax.text(x, event_y, event_details[i],
                    ha='center', va=va_event, fontsize=detail_fontsize,
                    color=label_color, bbox=bbox_props, zorder=6)
    
    # --- Appearance ---
    ax.set_aspect('equal', adjustable='box')
    padding_x = x_spacing * 0.8
    # Adjust vertical limits to ensure everything fits
    ax.set_xlim(x_coords[0] - padding_x, x_coords[-1] + padding_x) 
    all_y_coords = [main_node_y - 2.5, main_node_y + 2.5] # Default y range
    if event_details: # Adjust if event details boxes are present
         # Find min/max y based on potential box positions
         min_y_box = main_node_y - node_radius - box_offset - 0.5 # Estimate box height
         max_y_box = main_node_y + node_radius + box_offset + 0.5 # Estimate box height
         all_y_coords.extend([min_y_box, max_y_box])
    if node_labels: # Adjust if time labels are present
         min_y_label = main_node_y - node_radius - offset - 0.2
         max_y_label = main_node_y + node_radius + offset + 0.2
         all_y_coords.extend([min_y_label, max_y_label])
         
    ax.set_ylim(min(all_y_coords), max(all_y_coords)) # Set dynamic Y limits
    ax.axis('off')
    plt.tight_layout(pad=0.5) # Add slight padding
    
    # Create directory if needed
    dir_name = os.path.dirname(filename)
    if dir_name:
        os.makedirs(dir_name, exist_ok=True)
    
    plt.savefig(filename, dpi=300, bbox_inches='tight', facecolor=fig.get_facecolor())
    plt.close(fig)
    print(f"Roadmap visual saved as '{filename}'")

In [76]:
def create_day_roadmap(events, day_number, formatted_date, filename, symbol_folder): # Added symbol_folder
    """
    Creates a single-day roadmap image with symbols.
    Each node represents an event with an embedded symbol.
    The TIME is shown as the node label and the event details (name and location)
    appear in a text box on the opposite side.
    A day title (with the date) is drawn above the roadmap.
    """
    num_nodes = len(events)
    if num_nodes == 0:
        print(f"No events for day {day_number}, skipping.")
        return

    # Build node labels from event time and event details
    node_labels = [ev.get("time", "") for ev in events] # Use .get for safety
    event_details = [f"{ev.get('activity', 'N/A')}\n{ev.get('location', 'N/A')}" for ev in events]
    event_types = [ev.get("type") for ev in events] # Extract types for potential future use

    # Create the roadmap image for the day (save temporarily)
    temp_filename = f"roadmap_temp_day_{day_number}.png"
    generate_roadmap_visual(
        num_nodes=num_nodes, 
        node_labels=node_labels, 
        event_details=event_details, 
        filename=temp_filename,
        symbol_folder=symbol_folder, # Pass symbol folder path
        event_types=event_types       # Pass event types
    )
    
    # Open the generated roadmap image and add a day header (date)
    if not os.path.exists(temp_filename):
        print(f"Error: Temporary file {temp_filename} was not created.")
        return
        
    try:
        day_img = Image.open(temp_filename)
    except Exception as e:
        print(f"Error opening temporary image {temp_filename}: {e}")
        if os.path.exists(temp_filename):
             os.remove(temp_filename) # Clean up temp file if it exists but is corrupt
        return

    width, height = day_img.size
    header_height = 80 # Adjust as needed for font size
    # Use the background color from generate_roadmap_visual for consistency
    new_img = Image.new("RGB", (width, height + header_height), color='#E0E0E0') 
    draw = ImageDraw.Draw(new_img)
    
    try:
        # Ensure font path is correct or use a guaranteed default
        header_font = ImageFont.truetype("arial.ttf", 36) # Adjusted size
    except OSError:
        print("Arial font not found, using default PIL font.")
        header_font = ImageFont.load_default() # Fallback font
    
    # for writting the header_text for data
    # header_text = f"Day {day_number} - {formatted_date}"
    # # Use textbbox for more accurate centering with PIL > v8.0.0
    # try:
    #     header_bbox = draw.textbbox((0, 0), header_text, font=header_font)
    #     header_width = header_bbox[2] - header_bbox[0]
    #     header_x = (width - header_width) // 2
    #     header_y = 20 # Padding from top
    #     draw.text((header_x, header_y), header_text, font=header_font, fill='#6053D3') 
    # except AttributeError: # Fallback for older PIL versions without textbbox
    #      # Deprecated method, less accurate
    #      header_width, _ = draw.textsize(header_text, font=header_font) 
    #      header_x = (width - header_width) // 2
    #      header_y = 20 
    #      draw.text((header_x, header_y), header_text, font=header_font, fill='#6053D3')

    # new_img.paste(day_img, (0, header_height))
    new_img.paste(day_img)
    new_img.save(filename)
    
    # Clean up temporary file
    try:
        os.remove(temp_filename) 
    except OSError as e:
        print(f"Warning: Could not remove temporary file {temp_filename}. Error: {e}")
        
    print(f"Day {day_number} roadmap saved as '{filename}'")

In [68]:
def get_roadmap_filename(filename):
    road_maps_imgs = os.listdir(f"{os.getcwd()}/roadmap")
    if len(road_maps_imgs) == 0:
        return f'{filename}_00.png'
    
    if len(road_maps_imgs) == 1:
        return f'{filename}_01.png'

    last_two_elements = road_maps_imgs[-1][-6:-4]
    roadmap_num = int(last_two_elements) + 1

    return f'roadmap/{filename}_{roadmap_num:02}.png'

In [54]:
def main():
    try:
        # Consider getting key from environment variables for security
        openai_key = input("Enter your OpenAI API key: ") 
        if not openai_key:
             raise ValueError("OpenAI API key is required.")
        openai.api_key = openai_key
        
        user_inputs = collect_user_inputs()
        print("\nGenerating your sports event itinerary via OpenAI...")
        itinerary_text = generate_itinerary(user_inputs)
        
        print("\nGenerated Itinerary Text:")
        print(itinerary_text)
        
        # --- !!! IMPORTANT !!! ---
        # The following part assumes you have a function `parse_itinerary` 
        # and `create_multi_day_roadmap_images` or similar to process the 
        # 'itinerary_text' (which is a string) into the structured 'events' list 
        # expected by 'create_day_roadmap', and then call it for each day.
        # Since those functions are missing, this part is commented out.
        # You would need to implement parsing logic based on the format requested 
        # in the 'generate_itinerary' prompt.
        
        # Example (Conceptual - requires implementation):
        # daily_events = parse_itinerary(itinerary_text, user_inputs['start_date']) 
        # output_folder = "daily_roadmaps"
        # os.makedirs(output_folder, exist_ok=True)
        # for i, day_data in enumerate(daily_events):
        #     create_day_roadmap(
        #         events=day_data['events'], 
        #         day_number=i + 1, 
        #         formatted_date=day_data['date'].strftime('%Y-%m-%d'), 
        #         filename=os.path.join(output_folder, f"day_{i+1}_roadmap.png"),
        #         symbol_folder=SYMBOLS_FOLDER # Pass the symbol folder here
        #     )

        print("\nVisual roadmap generation skipped (requires itinerary parsing logic).")
        # print("\nYour sports event roadmap has been generated!")

    except ValueError as ve:
         print(f"Input Error: {ve}")
    except Exception as e:
        print(f"An error occurred during the main workflow: {e}")


In [77]:
# testing the roadmap
dummy_events_list = create_dummy_data()
test_day_number = 1
test_date = datetime.date.today().strftime("%Y-%m-%d")
output_filename = get_roadmap_filename("test_roadmap")

In [78]:
create_day_roadmap(
            events=dummy_events_list,
            day_number=test_day_number,
            formatted_date=test_date,
            filename=output_filename,
            symbol_folder=SYMBOLS_FOLDER # Pass the defined symbol folder path
        )

C:\Users\Admin\AppData\Local\Temp\ipykernel_509020\535351402.py:139: UserWarning: Setting the 'color' property will override the edgecolor or facecolor properties.
  small_circle = plt.Circle((x, small_y), small_circle_radius,


Roadmap visual saved as 'roadmap_temp_day_1.png'
Day 1 roadmap saved as 'roadmap/test_roadmap_07.png'
